In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [ ]:
# 1. Load Data
ptb_normal = pd.read_csv('ptbdb_normal.csv', header=None)
ptb_abnormal = pd.read_csv('ptbdb_abnormal.csv', header=None)

In [ ]:
#2. Preprocessing
# Combine and Shuffle
ptb_full = pd.concat([ptb_normal, ptb_abnormal], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

# Split features (0-186) and label (187)
X = ptb_full.iloc[:, :-1].values
y = ptb_full.iloc[:, -1].values

# Reshape for 1D CNN: [samples, time_steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# 3. Build 1D CNN Model
model = models.Sequential([
    layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(187, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 4. Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

In [ ]:
# 5. Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")